In [1]:
import torch
import torch.nn.functional as F

In [2]:
import platform

#운영체제에 따라 mecab설치 방법이 다름.
if platform.system() == "Windows":
    try:
        from eunjeon import Mecab
        
    except:
        print("please install eunjeon module")
else:  # Ubuntu일 경우
  try:
    from konlpy.tag import Mecab
  except:
    print("please install Mecab")



In [3]:
import pandas as pd

data = pd.read_csv(r'./lyrics_by_artist_and_age.csv')

### 1. 노래 가사를 문장 단위로 자르는 기준은?
줄넘김마다 끊기에는 너무 짧기도 하고 애매...


In [4]:
data.head()

,Unnamed: 0,lyrics,artist,age
0,0,나 아직 꿈을 꾸죠\n밤하늘 희미한 달빛처럼\n눈부셨던 기억 속에 그 사람\n어렴풋...,이선희,80
1,1,약속해요 이 순간이 다 지나고\n다시 보게 되는 그날\n모든 걸 버리고 그대 곁에 ...,이선희,80
2,2,소슬한 바람이 마음에 불어\n나비인 듯 날아 마음이 닿는 곳\n맘 같지 않은 세상\...,이선희,80
3,3,그렇게 대단한 운명까진\n바란적 없다 생각했는데\n그대 하나 떠나간 내 하룬 이제\...,이선희,80
4,4,이렇게 함께하는 편안한\n서로의 곁 연애할 때처럼 설레요\n밤이 내린 하늘엔 별이 ...,이선희,80


In [5]:
df = pd.DataFrame(data, columns=['lyrics','artist','age'])

In [6]:
df.head()

,lyrics,artist,age
0,나 아직 꿈을 꾸죠\n밤하늘 희미한 달빛처럼\n눈부셨던 기억 속에 그 사람\n어렴풋...,이선희,80
1,약속해요 이 순간이 다 지나고\n다시 보게 되는 그날\n모든 걸 버리고 그대 곁에 ...,이선희,80
2,소슬한 바람이 마음에 불어\n나비인 듯 날아 마음이 닿는 곳\n맘 같지 않은 세상\...,이선희,80
3,그렇게 대단한 운명까진\n바란적 없다 생각했는데\n그대 하나 떠나간 내 하룬 이제\...,이선희,80
4,이렇게 함께하는 편안한\n서로의 곁 연애할 때처럼 설레요\n밤이 내린 하늘엔 별이 ...,이선희,80


In [7]:
df_sentiment_train = pd.read_excel('./sentiment/Training/train.xlsx',engine='openpyxl')
df_sentiment_valid = pd.read_excel('./sentiment/Validation/valid.xlsx',engine='openpyxl')

In [8]:
df_sentiment_train.head()

,번호,연령,성별,상황키워드,신체질환,감정_대분류,감정_소분류,사람문장1,시스템응답1,사람문장2,시스템응답2,사람문장3,시스템응답3,사람문장4,시스템응답4
0,44164,청년,남성,"연애, 결혼, 출산",해당없음,기쁨,신이 난,아내가 드디어 출산하게 되어서 정말 신이 나.,아내분이 출산을 하시는군요. 정말 축하드려요.,아 지금 정말 신이 나.,잘 된 일이네요.,아기가 점점 클게 벌써 기대가 되네. 내가 많이 놀아줘야지.,좋은 아빠가 되실 거 같아요. 진심으로 축하드려요.,NaN,NaN
1,3926,노년,남성,"건강, 죽음",만성질환 유,불안,스트레스 받는,당뇨랑 합병증 때문에 먹어야 할 약이 열 가지가 넘어가니까 스트레스야.,약 종류가 많아 번거로우시겠어요.,건강할 때 관리 좀 잘할걸 하는 생각이 들더라고.,현재 상황에서 변화를 주기 위해 어떻게 하면 될까요?,약을 잘 챙겨 먹고 나을 수 있도록 노력해야지.,약을 잘 챙겨 먹어 건강해지시길 바랄게요.,NaN,NaN
2,50882,청소년,여성,학업 및 진로,해당없음,당황,당황,고등학교에 올라오니 중학교 때보다 수업이 갑자기 어려워져서 당황스러워.,고등학교 수업이 중학교 때와는 다르게 갑자기 어려워서 당황스러우시군요.,아직 학기 초인데 내가 수업에 잘 따라갈 수 있을지 걱정돼.,이 상황에서 할 수 있는 게 어떤 것이 있을까요?,중학교 수업을 들을 때보다 훨씬 더 열심히 할 거야.,수업을 어떤 방식으로 열심히 듣는게 좋을까요?,선생님이 강조하신 부분을 필기하고 집에서 매일 수업 내용을 복습하려고 해.,열심히 해서 좋은 결과가 있었으면 좋겠어요.
3,31303,노년,남성,재정,만성질환 무,기쁨,신이 난,재취업이 돼서 받게 된 첫 월급으로 온 가족이 외식을 할 예정이야. 너무 행복해.,재취업 후 첫 월급이라 정말 기쁘시겠어요.,퇴직 후 다시는 돈을 못 벌 줄 알았는데 이렇게 월급으로 가족에게 맛있는 밥을 살 ...,이 기분을 유지하려면 어떻게 해야 할까요?,회사생활을 열심히 해서 계속 월급을 받을거야!,계속 원하시는 대로 회사생활이 이어지길 바랄게요.,NaN,NaN
4,47200,노년,여성,재정,만성질환 유,기쁨,안도,빚을 드디어 다 갚게 되어서 이제야 안도감이 들어.,기분 좋으시겠어요. 앞으로는 어떻게 하실 생각이신가요?,빚도 다 갚았으니 당분간은 아무 생각도 안 하며 살고 싶어.,빚도 다 갚았으니 당분간은 아무 생각도 안 하며 살고 싶으시군요.,NaN,NaN,NaN,NaN


In [9]:
df_sentiment_train_ref = pd.DataFrame(df_sentiment_train,columns=['감정_대분류','사람문장1'])

In [10]:
df_sentiment_valid_ref = pd.DataFrame(df_sentiment_valid,columns=['감정_대분류','사람문장1'])

In [11]:
df_sentiment_train_ref.head()

,감정_대분류,사람문장1
0,기쁨,아내가 드디어 출산하게 되어서 정말 신이 나.
1,불안,당뇨랑 합병증 때문에 먹어야 할 약이 열 가지가 넘어가니까 스트레스야.
2,당황,고등학교에 올라오니 중학교 때보다 수업이 갑자기 어려워져서 당황스러워.
3,기쁨,재취업이 돼서 받게 된 첫 월급으로 온 가족이 외식을 할 예정이야. 너무 행복해.
4,기쁨,빚을 드디어 다 갚게 되어서 이제야 안도감이 들어.


감정 대분류 -> one-hot labeling

사람문장1 -> 영어로 column 만들기

In [12]:
df_sentiment_train_ref.rename(columns={'감정_대분류':'sentiment','사람문장1':'sent'},inplace=True)

In [13]:
df_sentiment_valid_ref.rename(columns={'감정_대분류':'sentiment','사람문장1':'sent'},inplace=True)

In [14]:
df_sentiment_train_ref.head()

,sentiment,sent
0,기쁨,아내가 드디어 출산하게 되어서 정말 신이 나.
1,불안,당뇨랑 합병증 때문에 먹어야 할 약이 열 가지가 넘어가니까 스트레스야.
2,당황,고등학교에 올라오니 중학교 때보다 수업이 갑자기 어려워져서 당황스러워.
3,기쁨,재취업이 돼서 받게 된 첫 월급으로 온 가족이 외식을 할 예정이야. 너무 행복해.
4,기쁨,빚을 드디어 다 갚게 되어서 이제야 안도감이 들어.


In [15]:
from sklearn.preprocessing import LabelEncoder

In [16]:
def strip_x(x):
    x = x.strip()
    return x
df_sentiment_train_ref.sentiment = df_sentiment_train_ref.sentiment.apply(strip_x)

In [17]:
le = LabelEncoder()
le.fit(df_sentiment_train_ref.sentiment)

LabelEncoder()

In [18]:
train_senti = le.transform(df_sentiment_train_ref.sentiment)

In [19]:
valid_senti = le.transform(df_sentiment_valid_ref.sentiment)

In [20]:
df_train_onehot = pd.concat([pd.DataFrame(df_sentiment_train_ref.sent),pd.DataFrame(train_senti)],axis=1)
df_train_onehot.head()

,sent,0
0,아내가 드디어 출산하게 되어서 정말 신이 나.,0
1,당뇨랑 합병증 때문에 먹어야 할 약이 열 가지가 넘어가니까 스트레스야.,3
2,고등학교에 올라오니 중학교 때보다 수업이 갑자기 어려워져서 당황스러워.,1
3,재취업이 돼서 받게 된 첫 월급으로 온 가족이 외식을 할 예정이야. 너무 행복해.,0
4,빚을 드디어 다 갚게 되어서 이제야 안도감이 들어.,0


In [21]:
df_valid_onehot = pd.concat([pd.DataFrame(df_sentiment_valid_ref.sent),pd.DataFrame(valid_senti)],axis=1)
df_valid_onehot.head()

,sent,0
0,요즘 부모님과 많이 부딪혀.,2
1,엄마가 결국 집을 나갔어. 너무 너무 슬퍼.,5
2,학교에서 한 친구를 괴롭히는 무리에게 그만하라고 했어.,3
3,이번에 팀장님이 간단한 조사 업무를 부탁하셨는데 내가 잘못 처리했어. 너무 절망적이야.,1
4,남편이 이혼할 때 위자료를 주지 않으려고 변호사를 고용했어.,2


In [22]:
df_train_onehot.rename(columns={0:'sentiment'},inplace=True)
df_valid_onehot.rename(columns={0:'sentiment'},inplace=True)

In [23]:
df_train_onehot.head()

,sent,sentiment
0,아내가 드디어 출산하게 되어서 정말 신이 나.,0
1,당뇨랑 합병증 때문에 먹어야 할 약이 열 가지가 넘어가니까 스트레스야.,3
2,고등학교에 올라오니 중학교 때보다 수업이 갑자기 어려워져서 당황스러워.,1
3,재취업이 돼서 받게 된 첫 월급으로 온 가족이 외식을 할 예정이야. 너무 행복해.,0
4,빚을 드디어 다 갚게 되어서 이제야 안도감이 들어.,0


In [24]:
df_valid_onehot.head()

,sent,sentiment
0,요즘 부모님과 많이 부딪혀.,2
1,엄마가 결국 집을 나갔어. 너무 너무 슬퍼.,5
2,학교에서 한 친구를 괴롭히는 무리에게 그만하라고 했어.,3
3,이번에 팀장님이 간단한 조사 업무를 부탁하셨는데 내가 잘못 처리했어. 너무 절망적이야.,1
4,남편이 이혼할 때 위자료를 주지 않으려고 변호사를 고용했어.,2


In [25]:
class Preprocessor:
    def __init__(self):
        self.stopWords = []
    def lyrics_extractor(self,lyrics):
        nouns = []
        mecab = Mecab()
        for i in lyrics.sent:
            sents = mecab.pos(i)
            result = []
            for word in sents:
                if word not in self.stopWords:
                    if word[1][0] == 'N' or word[1][0] == 'V':
                        result.append(word[0])
            nouns.append(result)
        return nouns

In [26]:
prep = Preprocessor()
df_train = prep.lyrics_extractor(df_train_onehot)

In [27]:
df_valid = prep.lyrics_extractor(df_valid_onehot)

In [28]:
df_train_onehot.sent = df_train

In [29]:
df_train_onehot.head()

,sent,sentiment
0,"[아내, 출산, 되, 신, 나]",0
1,"[당뇨, 합병증, 때문, 먹, 할, 이, 열, 가지, 넘어가, 스트레스, 야]",3
2,"[고등학교, 올라오, 중학교, 때, 수업, 어려워져서, 당황]",1
3,"[재취업, 돼서, 받, 된, 월급, 온, 가족, 외식, 할, 예정, 이, 행복]",0
4,"[빚, 갚, 되, 안도감, 들]",0


In [30]:
df_valid_onehot.sent = df_valid

In [31]:
df_train_onehot.to_csv('./preprocessed/train.tsv',sep='\t',index=False)
df_valid_onehot.to_csv('./preprocessed/valid.tsv',sep='\t',index=False)

In [32]:
df.head()

,lyrics,artist,age
0,나 아직 꿈을 꾸죠\n밤하늘 희미한 달빛처럼\n눈부셨던 기억 속에 그 사람\n어렴풋...,이선희,80
1,약속해요 이 순간이 다 지나고\n다시 보게 되는 그날\n모든 걸 버리고 그대 곁에 ...,이선희,80
2,소슬한 바람이 마음에 불어\n나비인 듯 날아 마음이 닿는 곳\n맘 같지 않은 세상\...,이선희,80
3,그렇게 대단한 운명까진\n바란적 없다 생각했는데\n그대 하나 떠나간 내 하룬 이제\...,이선희,80
4,이렇게 함께하는 편안한\n서로의 곁 연애할 때처럼 설레요\n밤이 내린 하늘엔 별이 ...,이선희,80


'\n' 구분자로 끊기

In [33]:
df['lyric_sep'] = df['lyrics'].str.split('\n')

In [34]:
s = df['lyric_sep'][0]

In [35]:
def joinByN(n,lyric_sep):
    length = len(lyric_sep)
    lyrics = []
    for i in range(length//n):
        if (i*4+n)>length:
            j = length
        else: j=i*4+n
        lyrics.append(" ".join(lyric_sep[i*4:j]))
    return lyrics

In [36]:
joinByN(4,s)

['나 아직 꿈을 꾸죠 밤하늘 희미한 달빛처럼 눈부셨던 기억 속에 그 사람 어렴풋이 생각이 나네요',
 '바람에 흩날리는 꽃잎에 따뜻한 그대 향기를 느끼고 스쳐 지나간 듯한 짧았었던 인연이 이제',
 '내 전부인 걸요 같은 하늘 아래 그대와 함께 있다는 걸 지워질까 두려운 거죠',
 '푸른 바다 수평선까지 걸을 수 있다면 나 그대 손 놓지 않을게 하늘에 수놓아진 별빛에',
 '따뜻한 그대 숨결을 느끼고 운명처럼 그대와 이대로만 이렇게 남아 숨 쉴 수 있다면',
 '같은 하늘 아래 그대와 함께 있다는 걸 지워질까 두려운 거죠 푸른 바다 수평선까지',
 '걸을 수 있다면 나 그대 손 놓지 않을게 같은 하늘 아래 그대와 함께 있다는 걸',
 '지워질까 두려운 거죠 푸른 바다 수평선까지 걸을 수 있다면 나 그대 손 놓지 않을게']

In [37]:
df['lyrics_sep'] = df['lyric_sep'].apply(lambda x : joinByN(4,x))

In [38]:
df['lyrics_sep'][0]

['나 아직 꿈을 꾸죠 밤하늘 희미한 달빛처럼 눈부셨던 기억 속에 그 사람 어렴풋이 생각이 나네요',
 '바람에 흩날리는 꽃잎에 따뜻한 그대 향기를 느끼고 스쳐 지나간 듯한 짧았었던 인연이 이제',
 '내 전부인 걸요 같은 하늘 아래 그대와 함께 있다는 걸 지워질까 두려운 거죠',
 '푸른 바다 수평선까지 걸을 수 있다면 나 그대 손 놓지 않을게 하늘에 수놓아진 별빛에',
 '따뜻한 그대 숨결을 느끼고 운명처럼 그대와 이대로만 이렇게 남아 숨 쉴 수 있다면',
 '같은 하늘 아래 그대와 함께 있다는 걸 지워질까 두려운 거죠 푸른 바다 수평선까지',
 '걸을 수 있다면 나 그대 손 놓지 않을게 같은 하늘 아래 그대와 함께 있다는 걸',
 '지워질까 두려운 거죠 푸른 바다 수평선까지 걸을 수 있다면 나 그대 손 놓지 않을게']

In [39]:
df.drop('lyric_sep',axis=1,inplace=True)

In [40]:
df.head()

,lyrics,artist,age,lyrics_sep
0,나 아직 꿈을 꾸죠\n밤하늘 희미한 달빛처럼\n눈부셨던 기억 속에 그 사람\n어렴풋...,이선희,80,[나 아직 꿈을 꾸죠 밤하늘 희미한 달빛처럼 눈부셨던 기억 속에 그 사람 어렴풋이 ...
1,약속해요 이 순간이 다 지나고\n다시 보게 되는 그날\n모든 걸 버리고 그대 곁에 ...,이선희,80,[약속해요 이 순간이 다 지나고 다시 보게 되는 그날 모든 걸 버리고 그대 곁에 서...
2,소슬한 바람이 마음에 불어\n나비인 듯 날아 마음이 닿는 곳\n맘 같지 않은 세상\...,이선희,80,[소슬한 바람이 마음에 불어 나비인 듯 날아 마음이 닿는 곳 맘 같지 않은 세상 그...
3,그렇게 대단한 운명까진\n바란적 없다 생각했는데\n그대 하나 떠나간 내 하룬 이제\...,이선희,80,[그렇게 대단한 운명까진 바란적 없다 생각했는데 그대 하나 떠나간 내 하룬 이제 운...
4,이렇게 함께하는 편안한\n서로의 곁 연애할 때처럼 설레요\n밤이 내린 하늘엔 별이 ...,이선희,80,[이렇게 함께하는 편안한 서로의 곁 연애할 때처럼 설레요 밤이 내린 하늘엔 별이 가...


In [103]:
def lyrics_extractor(lyrics):
    nouns = []
    mecab = Mecab()
    for sentence in lyrics.lyrics_sep:
        l_sent = []
        for i in sentence:
            sents = mecab.pos(i)
            result = []
            for word in sents:
                if word[1][0] == 'N' or word[1][0] == 'V':
                    result.append(word[0])
            if len(result)>0:
                l_sent.append(result)
        nouns.append(l_sent)
    return nouns

In [85]:
def drop_if_eng(df):
    df['one_string'] = df['lyrics_sep'].str.join(" ")
    mecab=Mecab()
    for i in df.one_string:
        sents = mecab.pos(i)
        n = 0
        result = []
        for word in sents:
            if word[1] == 'SL':
                n+=1
        if n>(len(sents)//2):
            df=df[df.one_string != i]
            print(len(df))
    return df

In [86]:
len(df)

1342

In [87]:
df_saved = drop_if_eng(df)


1340
1338
1336
1334
1332
1330
1328
1326
1324
1322
1320
1318
1316
1314
1312
1310
1308
1306
1304
1302
1301
1300
1299
1298
1297
1296
1295
1294
1293
1292
1291
1290
1289
1288
1287
1286
1285
1284
1283
1282
1281
1280
1279
1278
1277
1276
1275
1274
1273
1272
1271
1270
1269
1268
1267
1266
1265
1264
1263
1262
1261
1260
1259
1258
1257
1256
1255
1254
1253
1252
1251
1250
1249
1248
1247
1246
1246
1246
1246
1246
1246
1246
1246
1246
1246
1246
1246
1246
1246
1246
1246
1246
1246
1246
1246
1246
1245
1244
1243
1242
1241
1240
1239
1238
1237
1236
1235
1234
1233
1232
1231
1230
1229
1228
1227
1226
1225
1224
1223
1222
1221
1220
1219
1218
1217
1216
1215
1214
1213
1212
1211
1210
1209
1208
1207
1206
1205
1204
1203
1202
1201
1200
1199
1198
1197
1196
1195
1194
1193
1192
1191
1190
1189
1188
1187
1186
1185
1184
1183
1182
1181
1180
1179
1178
1177
1176
1175
1174
1173
1172
1171
1170
1169
1168
1167
1166
1165
1164
1163
1162
1161
1160
1159
1158
1157
1156
1155
1154
1153
1152
1151
1150
1149
1148
1147
1146
1145
1144
1143
1142


In [104]:
df_saved

,lyrics,artist,age,lyrics_sep,one_string,morphs
0,나 아직 꿈을 꾸죠\n밤하늘 희미한 달빛처럼\n눈부셨던 기억 속에 그 사람\n어렴풋...,이선희,80,[나 아직 꿈을 꾸죠 밤하늘 희미한 달빛처럼 눈부셨던 기억 속에 그 사람 어렴풋이 ...,나 아직 꿈을 꾸죠 밤하늘 희미한 달빛처럼 눈부셨던 기억 속에 그 사람 어렴풋이 생...,"[[나, 꿈, 꾸, 밤하늘, 달빛, 눈부셨, 기억, 속, 사람, 생각, 나, 네요]..."
1,약속해요 이 순간이 다 지나고\n다시 보게 되는 그날\n모든 걸 버리고 그대 곁에 ...,이선희,80,[약속해요 이 순간이 다 지나고 다시 보게 되는 그날 모든 걸 버리고 그대 곁에 서...,약속해요 이 순간이 다 지나고 다시 보게 되는 그날 모든 걸 버리고 그대 곁에 서서...,"[[약속, 순간, 지나, 보, 되, 그날, 걸, 버리, 그대, 곁, 서, 남, 길,..."
2,소슬한 바람이 마음에 불어\n나비인 듯 날아 마음이 닿는 곳\n맘 같지 않은 세상\...,이선희,80,[소슬한 바람이 마음에 불어 나비인 듯 날아 마음이 닿는 곳 맘 같지 않은 세상 그...,소슬한 바람이 마음에 불어 나비인 듯 날아 마음이 닿는 곳 맘 같지 않은 세상 그 ...,"[[바람, 마음, 불, 나비, 인, 듯, 날, 마음, 닿, 곳, 맘, 같, 않, 세..."
3,그렇게 대단한 운명까진\n바란적 없다 생각했는데\n그대 하나 떠나간 내 하룬 이제\...,이선희,80,[그렇게 대단한 운명까진 바란적 없다 생각했는데 그대 하나 떠나간 내 하룬 이제 운...,그렇게 대단한 운명까진 바란적 없다 생각했는데 그대 하나 떠나간 내 하룬 이제 운명...,"[[운명, 바란, 적, 없, 생각, 그대, 하나, 떠나간, 하룬, 운명, 아님, 채..."
4,이렇게 함께하는 편안한\n서로의 곁 연애할 때처럼 설레요\n밤이 내린 하늘엔 별이 ...,이선희,80,[이렇게 함께하는 편안한 서로의 곁 연애할 때처럼 설레요 밤이 내린 하늘엔 별이 가...,이렇게 함께하는 편안한 서로의 곁 연애할 때처럼 설레요 밤이 내린 하늘엔 별이 가득...,"[[하, 편안, 서로, 곁, 연애, 때, 설레, 밤, 내린, 하늘, 별], [우리,..."
...,...,...,...,...,...,...
1337,다가서면 멀어지는\n저 꿈결처럼\n그댄 어디로 가나요\n내 맘속에 살고 있는 그...,청하,10,[다가서면 멀어지는 저 꿈결처럼 그댄 어디로 가나요 내 맘속에 살고 있는 그대란...,다가서면 멀어지는 저 꿈결처럼 그댄 어디로 가나요 내 맘속에 살고 있는 그대란 ...,"[[다가서, 멀, 지, 꿈결, 그댄, 어디, 가, 내, 맘속, 살, 있, 그대, 란..."
1338,왠지 설레는 느낌\nMake me feel so high\n환한 미소가\n(I re...,청하,10,[왠지 설레는 느낌 Make me feel so high 환한 미소가 (I real...,왠지 설레는 느낌 Make me feel so high 환한 미소가 (I reall...,"[[설레, 느낌, 환한, 미소], [빠져들, 싶, 너, 담, 둘, 래], [이젠, ..."
1339,어디선가 넌 나를 지켜보겠지\n지키지 못한 약속 모두 다 잊은 채\n이제야 나는 제...,청하,10,[어디선가 넌 나를 지켜보겠지 지키지 못한 약속 모두 다 잊은 채 이제야 나는 제자...,어디선가 넌 나를 지켜보겠지 지키지 못한 약속 모두 다 잊은 채 이제야 나는 제자리...,"[[어디, 넌, 나, 지켜보, 지키, 못한, 약속, 잊, 채, 나, 제자리, 찾아온..."
1340,"You can find me, right?\nYou can find me, righ...",청하,10,"[You can find me, right? You can find me, righ...","You can find me, right? You can find me, right...","[[], [어지러운, 날, 반, 가린, 돌아가, 시선, 가두, 저, 다르, 바래, ..."


In [105]:
df_saved_morphs = lyrics_extractor(df_saved)

In [106]:
df_saved['morphs'] = df_saved_morphs

In [109]:
df_saved['morphs'][0]

[['나', '꿈', '꾸', '밤하늘', '달빛', '눈부셨', '기억', '속', '사람', '생각', '나', '네요'],
 ['바람', '흩날리', '꽃잎', '그대', '향기', '느끼', '스쳐', '지나간', '듯', '짧', '인연'],
 ['내',
  '전부',
  '인',
  '걸요',
  '같',
  '하늘',
  '아래',
  '그대',
  '있',
  '걸',
  '지워',
  '질까',
  '두려운',
  '거',
  '죠'],
 ['푸른',
  '바',
  '수평선',
  '걸',
  '수',
  '있',
  '나',
  '그대',
  '손',
  '놓',
  '않',
  '게',
  '하늘',
  '수놓',
  '진',
  '별빛'],
 ['그대', '숨결', '느끼', '운명', '그대', '남', '숨', '쉴', '수', '있'],
 ['같',
  '하늘',
  '아래',
  '그대',
  '있',
  '걸',
  '지워',
  '질까',
  '두려운',
  '거',
  '죠',
  '푸른',
  '바',
  '수평선'],
 ['걸',
  '수',
  '있',
  '나',
  '그대',
  '손',
  '놓',
  '않',
  '게',
  '같',
  '하늘',
  '아래',
  '그대',
  '있',
  '걸'],
 ['지워',
  '질까',
  '두려운',
  '거',
  '죠',
  '푸른',
  '바',
  '수평선',
  '걸',
  '수',
  '있',
  '나',
  '그대',
  '손',
  '놓',
  '않',
  '게']]

In [108]:
df_saved.head()

,lyrics,artist,age,lyrics_sep,one_string,morphs
0,나 아직 꿈을 꾸죠\n밤하늘 희미한 달빛처럼\n눈부셨던 기억 속에 그 사람\n어렴풋...,이선희,80,[나 아직 꿈을 꾸죠 밤하늘 희미한 달빛처럼 눈부셨던 기억 속에 그 사람 어렴풋이 ...,나 아직 꿈을 꾸죠 밤하늘 희미한 달빛처럼 눈부셨던 기억 속에 그 사람 어렴풋이 생...,"[[나, 꿈, 꾸, 밤하늘, 달빛, 눈부셨, 기억, 속, 사람, 생각, 나, 네요]..."
1,약속해요 이 순간이 다 지나고\n다시 보게 되는 그날\n모든 걸 버리고 그대 곁에 ...,이선희,80,[약속해요 이 순간이 다 지나고 다시 보게 되는 그날 모든 걸 버리고 그대 곁에 서...,약속해요 이 순간이 다 지나고 다시 보게 되는 그날 모든 걸 버리고 그대 곁에 서서...,"[[약속, 순간, 지나, 보, 되, 그날, 걸, 버리, 그대, 곁, 서, 남, 길,..."
2,소슬한 바람이 마음에 불어\n나비인 듯 날아 마음이 닿는 곳\n맘 같지 않은 세상\...,이선희,80,[소슬한 바람이 마음에 불어 나비인 듯 날아 마음이 닿는 곳 맘 같지 않은 세상 그...,소슬한 바람이 마음에 불어 나비인 듯 날아 마음이 닿는 곳 맘 같지 않은 세상 그 ...,"[[바람, 마음, 불, 나비, 인, 듯, 날, 마음, 닿, 곳, 맘, 같, 않, 세..."
3,그렇게 대단한 운명까진\n바란적 없다 생각했는데\n그대 하나 떠나간 내 하룬 이제\...,이선희,80,[그렇게 대단한 운명까진 바란적 없다 생각했는데 그대 하나 떠나간 내 하룬 이제 운...,그렇게 대단한 운명까진 바란적 없다 생각했는데 그대 하나 떠나간 내 하룬 이제 운명...,"[[운명, 바란, 적, 없, 생각, 그대, 하나, 떠나간, 하룬, 운명, 아님, 채..."
4,이렇게 함께하는 편안한\n서로의 곁 연애할 때처럼 설레요\n밤이 내린 하늘엔 별이 ...,이선희,80,[이렇게 함께하는 편안한 서로의 곁 연애할 때처럼 설레요 밤이 내린 하늘엔 별이 가...,이렇게 함께하는 편안한 서로의 곁 연애할 때처럼 설레요 밤이 내린 하늘엔 별이 가득...,"[[하, 편안, 서로, 곁, 연애, 때, 설레, 밤, 내린, 하늘, 별], [우리,..."
